In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [3]:
df = pd.read_csv("earthquake_alert_balanced_dataset.csv")

df = df.dropna().reset_index(drop=True)

features = ['magnitude', 'depth', 'cdi', 'mmi', 'sig']
target = 'alert'

target_encoder = LabelEncoder()
target_encoder.fit(df[target])
target_encoder.classes_ = np.array(['green', 'yellow', 'orange', 'red'])
df[target] = target_encoder.transform(df[target])

print("Dataset shape:", df.shape)
df.head()

Dataset shape: (1300, 6)


,magnitude,depth,cdi,mmi,sig,alert
0,7.0,14.0,8.0,7.0,0.0,0
1,6.9,25.0,4.0,4.0,-33.0,0
2,7.0,579.0,3.0,3.0,-13.0,0
3,7.3,37.0,5.0,5.0,65.0,0
4,6.6,624.0,0.0,2.0,-98.0,0


In [4]:
def alert0_cond(cdi, mmi):
    def model_func(X, a,b,c,d,e,f,g,h,i):
        cdi, mmi = X
        return (a*cdi+b*mmi+i)/(1+np.exp(-(c*cdi+d*mmi+e*cdi**2+f*mmi**2+g*cdi*mmi+h)))
    pms = [1.27984035e-02,  1.57367417e-02, -2.48405223e+00, -3.30812036e+00,
           2.02357722e-01,  1.49130611e-01, -7.33127329e-02,  2.42667445e+01,
        9.19931899e-01]
    return model_func((cdi, mmi), *pms) >= 0.7  # threshold at 0.7


In [ ]:
def mmi7_cond(magnitude, sig):
    def model_func(X, a,b,c,d,e,f,g,h,i):
        x, y = X
        return (a*x+b*y+i)/(1+np.exp(-(c*x+d*y+e*x**2+f*y**2+g*x*y+h)))
    pms =  [ 1.08530024e+02, -5.18588843e-01, -7.43785453e+00,  1.02482995e-01,
            3.94530496e-01, -1.37585339e-05, -1.31901142e-02,  2.90637996e+01,
            -6.84288504e+02]
    return model_func((magnitude, sig), *pms) >= 1.325 

In [43]:
def mmi8_cond(magnitude, sig):
    def model_func(X, a,b,c,d,e,f,g,h,i,j):
        x, y = X
        return ((-(x-a)*np.sin(c)+(y-b)*np.cos(c))-d*np.tanh(e*((x-a)*np.sin(c)+(y-b)*np.cos(c)))+i)/(1+np.exp(-(f*x+g*y+h)))
    pms =  [-3.12012780e+02, -3.41873745e+02, -9.15709217e-01, -1.19836149e+01,
            8.50046469e+00,  4.58161604e-02, -7.13311410e-03, -3.82951563e+00,
            -3.57056423e+02,  0.00000000e+00]
    return model_func((magnitude, sig), *pms) >= 2.5

In [ ]:
def if_classifier(x):
    magnitude, depth, cdi, mmi, sig, _ = x
    if alert0_cond(cdi, mmi):
        return 0
    
    if sig>100 or mmi==6 or (mmi==7 and (cdi<=6)):
        return 1
    
    if (mmi==7 and not mmi7_cond(magnitude, sig)):
        return 1
    
    if (mmi==7 and mmi7_cond(magnitude, sig)) or (mmi==8 and not mmi8_cond(magnitude, sig)):
        return 2
    
    if (mmi==8 and mmi8_cond(magnitude, sig)) or mmi==9:
        return 3
        
    return 0

In [65]:
# now compute the multiclass metrics (acc, precision, recall, f1) for ground_truth alert and predicted alert from if_classifier on the whole df
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
y_true = df['alert']
y_pred = np.array([if_classifier(x) for x in df.values]).astype(int)
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

Accuracy: 0.7285, Precision: 0.7565, Recall: 0.7285, F1-score: 0.7299


In [66]:
# now compute the confusion matrix and per-class metrics (acc, precision, recall, f1) for ground_truth alert and predicted alert from if_classifier on the whole df
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)
report = classification_report(y_true, y_pred, target_names=target_encoder.classes_, zero_division=0)
print("Classification Report:")
print(report)

Confusion Matrix:
[[219  83  17   6]
 [  1 233  60  31]
 [  0   7 207 111]
 [  0  27  10 288]]
Classification Report:
              precision    recall  f1-score   support

       green       1.00      0.67      0.80       325
      yellow       0.67      0.72      0.69       325
      orange       0.70      0.64      0.67       325
         red       0.66      0.89      0.76       325

    accuracy                           0.73      1300
   macro avg       0.76      0.73      0.73      1300
weighted avg       0.76      0.73      0.73      1300

